# Langchain

Tutorial criando uma LLM chain basic, 

In [7]:
from langchain_cohere import ChatCohere

llm = ChatCohere()
llm.invoke("how can langsmith help with testing?")


AIMessage(content="Langsmith can help with testing in a number of ways:\n\n1. **Automated Testing**: Langsmith can be used to automate the testing process. You can use Langsmith to write test cases and then use its natural language processing capabilities to automatically generate test data and test scripts. This can save a lot of time and effort in the testing process.\n\n2. **Natural Language Understanding**: Langsmith's advanced natural language understanding capabilities can be leveraged to create more effective test cases. You can use natural language to describe the test scenarios and Langsmith can understand and interpret these scenarios to generate the appropriate test cases.\n\n3. **Test Data Generation**: Langsmith can generate test data that is more realistic and relevant to your specific use case. By using natural language processing, Langsmith can understand the context and generate data that is meaningful and useful for testing.\n\n4. **Test Coverage Analysis**: Langsmith

In [9]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
])
chain = prompt | llm 
chain.invoke({"input": "how can langsmith help with testing?"})


AIMessage(content="Langsmith is a powerful tool that can greatly assist with testing and quality assurance processes, especially for software products with a user interface or natural language processing capabilities. Here's how Langsmith can help with testing:\n\n1. **User Interface Testing:**\n   - Langsmith can generate a variety of text inputs, including random words, phrases, and sentences, which can be used to test the robustness of user interfaces. By inputting these generated texts into different UI fields, you can identify any potential issues, such as text overflow, improper formatting, or unexpected behavior.\n   - You can use Langsmith to create targeted test cases. For example, you can generate specific text patterns, like long words, special characters, or localized content, to ensure that the user interface handles these cases gracefully.\n\n2. **Functional Testing:**\n   - Langsmith's text generation capabilities can be leveraged to create diverse test data. By generati

In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
])
output_parser = StrOutputParser()


chain = prompt | llm | 
chain.invoke({"input": "how can langsmith help with testing?"})


"Langsmith is a powerful tool that can greatly assist with testing and quality assurance processes, especially for software products with a user interface or natural language processing capabilities. Here's how Langsmith can help with testing:\n\n1. **User Interface Testing:**\n   - Internationalization and Localization: Langsmith enables you to easily translate and localize your user interface, making it possible to test your software in different languages. This helps ensure that your product works as intended for global audiences.\n   - Pseudo-Localization: Langsmith can generate pseudo-localized text, which replaces your original text with character sets and text lengths typical of a particular language. This helps test the layout and functionality of your user interface without the need for actual translations.\n\n2. **Functional Testing:**\n   - Translation Accuracy: Langsmith's translation capabilities can be leveraged to test the accuracy of translations in your software. By tr

## segunda parte do tutorial langchain

In [26]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_cohere.embeddings import CohereEmbeddings

from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain

from langchain_core.documents import Document
from langchain.chains import create_retrieval_chain


# carega as informações do site oficial do langchain
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
docs = loader.load()
#print(docs)

embeddings = CohereEmbeddings()

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})





'Langsmith can help with testing by visualizing test results.'

In [27]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_cohere.embeddings import CohereEmbeddings

from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain

from langchain_core.documents import Document
from langchain.chains import create_retrieval_chain


# carega as informações do site oficial do langchain
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
docs = loader.load()
#print(docs)

embeddings = CohereEmbeddings()

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)


retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])



LangSmith is a platform that aids in the development, monitoring, and testing of LLM applications. It offers a range of features to support testing, including:

- Initial Test Set: LangSmith enables developers to create datasets with input and reference output pairs, facilitating the running of tests on LLM applications. These test cases can be created, uploaded in bulk, or exported from application traces.
- Comparison View: This feature allows users to compare the performance of different application versions by evaluating their results on the same data points side-by-side. It helps identify regressions and improvements across multiple revisions.
- Beta Testing: LangSmith supports beta testing by capturing user feedback and allowing the annotation of traces. Feedback scores can be attached to logged traces, helping identify problematic responses. Annotating traces with respect to specific criteria provides insights into the application's performance and helps catch regressions.
- Add

## criando um chatbot

In [31]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage


# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})



{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content='Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookThis is outdated documentation for ğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith, which is no longer actively maintained.For up-to-date documentation, see the latest version.User GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, weâ€™ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if theyâ€™re just starting their journey.Prototypingâ€‹Prototyping LLM applications often involves quick experim

In [31]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})




{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content='Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookThis is outdated documentation for ğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith, which is no longer actively maintained.For up-to-date documentation, see the latest version.User GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, weâ€™ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if theyâ€™re just starting their journey.Prototypingâ€‹Prototyping LLM applications often involves quick experim